In [1]:
import os
os.chdir("/app")

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from torchinfo import summary
from IPython.display import display, Markdown
import time

device_map = {"": 0} if torch.cuda.is_available() else {"": "cpu"}
# Define custom load function
def load_custom_model(model_dir):
    model = AutoModelForCausalLM.from_pretrained(model_dir, device_map=device_map)
    tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
    return model, tokenizer

# Define chat function
def chat(model, tokenizer, message, max_new_tokens=1000):
    messages = []
    
    messages.append({'role': 'user', 'content': message})
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt", truncation=True, max_length=model.config.max_position_embeddings).to(model.device)

    with torch.no_grad():
        outputs = model.generate(inputs, max_new_tokens=max_new_tokens, do_sample=False, top_k=50, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)

    model_response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    messages.append({'role': 'system', 'content': model_response})

    return model_response

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "Qwen/Qwen2.5-Coder-3B-Instruct-angular-trained-merged"
trained_model, trained_tokenizer = load_custom_model(model_name)

In [3]:
prompt = "Create an Angular component named `UserProfileComponent` to display `username`, `email`, and `age` with TypeScript, HTML, and CSS."

start_time = time.time()
trained_model_response = chat(trained_model, trained_tokenizer, prompt, max_new_tokens=2000)
end_time = time.time()

time_taken = end_time - start_time

display(Markdown(f"**User Prompt:** \n\n {prompt}"))
display(Markdown(f"**{model_name} Response:** \n\n {trained_model_response}"))
display(Markdown(f"**Time taken:** {time_taken} seconds"))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


**User Prompt:** 

 Create an Angular component named `UserProfileComponent` to display `username`, `email`, and `age` with TypeScript, HTML, and CSS.

**Qwen/Qwen2.5-Coder-3B-Instruct-angular-trained-merged Response:** 

 To create an Angular component named `UserProfileComponent`, you can follow these steps:

1. Open your terminal and navigate to the root directory of your Angular project.
2. Run the following command to generate the component:
   ```
   ng generate component user-profile
   ```
3. This command creates the following files in the `src/app/user-profile/` directory:
   * `user-profile.component.ts`
   * `user-profile.component.html`
   * `user-profile.component.css`
4. Open the `user-profile.component.ts` file and add the following code to define the component's properties:
   ```
   import { Component } from '@angular/core';
   
   @Component({
     selector: 'app-user-profile',
     templateUrl: './user-profile.component.html',
     styleUrls: ['./user-profile.component.css']
   })
   export class UserProfileComponent {
     username = 'John Doe';
     email = 'john.doe@example.com';
     age = 30;
   }
   ```
5. Open the `user-profile.component.html` file and add the following code to display the component's properties:
   ```
   <h2>User Profile</h2>
   <p>Username: {{ username }}</p>
   <p>Email: {{ email }}</p>
   <p>Age: {{ age }}</p>
   ```
6. Open the `user-profile.component.css` file and add the following code to style the component:
   ```
   h2 {
     color: #333;
   }
   
   p {
     margin: 10px 0;
   }
   ```
7. Open the `app.component.html` file and add the following code to include the `UserProfileComponent` in the application:
   ```
   <app-user-profile></app-user-profile>
   ```
8. Save all changes.

Now, when you run the application, it displays the `UserProfileComponent` on the page. The component displays the `username`, `email`, and `age` properties that you defined in the `UserProfileComponent` class.

**Time taken:** 6.0183141231536865 seconds

In [4]:
import time
import json
import os
from tqdm import tqdm
from IPython.display import display, Markdown

# List of prompts
prompts = [
    "Create an Angular component named `UserProfileComponent` to display `username`, `email`, and `age` with TypeScript, HTML, and CSS.",
    "Write an Angular service `UserService` with methods for fetching all users (GET `/api/users`), fetching user by ID (GET `/api/users/{id}`), and creating a new user (POST `/api/users`).",
    "Using Angular generate `ProductFormComponent` with a Reactive Form for `name`, `price`, and `description`, including validation and form submission handling.",
    "Create a paginated, sortable Angular Material table for `id`, `name`, `department`, and `salary`, with a search bar for filtering by name.",
    "Set up Angular routing with `/dashboard`, `/users`, `/users/:id`, and redirect other paths to `/dashboard`, including lazy loading for `UsersModule` and guards for `/users`."
]


# Directory to save responses
response_dir = "prompt_responses"
os.makedirs(response_dir, exist_ok=True)

# Loop through prompts and generate responses with progress bar
for i, prompt in enumerate(tqdm(prompts, desc="Generating responses")):
    start_time = time.time()
    trained_model_response = chat(trained_model, trained_tokenizer, prompt, max_new_tokens=2000)
    end_time = time.time()
    
    time_taken = end_time - start_time
    
    response_data = {
        "prompt": prompt,
        "response": trained_model_response,
        "time_taken": time_taken
    }
    
    # Save response to JSON file
    with open(os.path.join(response_dir, f"response_{i+1}.json"), "w") as f:
        json.dump(response_data, f)

# Function to read and display response in Markdown
def read_response(index):
    try:
        with open(os.path.join(response_dir, f"response_{index}.json"), "r") as f:
            response_data = json.load(f)
        
        display(Markdown(f"**User Prompt:** \n\n {response_data['prompt']}"))
        display(Markdown(f"**Model Response:** \n\n {response_data['response']}"))
        display(Markdown(f"**Time taken:** {response_data['time_taken']} seconds"))
    except FileNotFoundError:
        print(f"Response file for index {index} not found.")

# Example usage
read_response(1)

Generating responses: 100%|██████████| 5/5 [00:56<00:00, 11.33s/it]


**User Prompt:** 

 Create an Angular component named `UserProfileComponent` to display `username`, `email`, and `age` with TypeScript, HTML, and CSS.

**Model Response:** 

 To create an Angular component named `UserProfileComponent`, you can follow these steps:

1. Open your terminal and navigate to the root directory of your Angular project.
2. Run the following command to generate the component:
   ```
   ng generate component user-profile
   ```
3. This command creates the following files in the `src/app/user-profile/` directory:
   * `user-profile.component.ts`
   * `user-profile.component.html`
   * `user-profile.component.css`
4. Open the `user-profile.component.ts` file and add the following code to define the component's properties:
   ```
   import { Component } from '@angular/core';
   
   @Component({
     selector: 'app-user-profile',
     templateUrl: './user-profile.component.html',
     styleUrls: ['./user-profile.component.css']
   })
   export class UserProfileComponent {
     username = 'John Doe';
     email = 'john.doe@example.com';
     age = 30;
   }
   ```
5. Open the `user-profile.component.html` file and add the following code to display the component's properties:
   ```
   <h2>User Profile</h2>
   <p>Username: {{ username }}</p>
   <p>Email: {{ email }}</p>
   <p>Age: {{ age }}</p>
   ```
6. Open the `user-profile.component.css` file and add the following code to style the component:
   ```
   h2 {
     color: #333;
   }
   
   p {
     margin: 10px 0;
   }
   ```
7. Open the `app.component.html` file and add the following code to include the `UserProfileComponent` in the application:
   ```
   <app-user-profile></app-user-profile>
   ```
8. Save all changes.

Now, when you run the application, it displays the `UserProfileComponent` on the page. The component displays the `username`, `email`, and `age` properties that you defined in the `UserProfileComponent` class.

**Time taken:** 5.803786516189575 seconds

In [5]:
read_response(3)

**User Prompt:** 

 Using Angular generate `ProductFormComponent` with a Reactive Form for `name`, `price`, and `description`, including validation and form submission handling.

**Model Response:** 

 To create a `ProductFormComponent` using Angular's Reactive Forms, you can follow these steps:

1. **Generate the Component:**

   First, generate the component using the Angular CLI.

   ```bash
   ng generate component product-form
   ```

2. **Import Necessary Modules:**

   Open the generated `product-form.component.ts` file and import the necessary modules from Angular's forms module.

   ```typescript
   import { Component } from '@angular/core';
   import { FormBuilder, FormGroup, Validators } from '@angular/forms';
   ```

3. **Create the Form:**

   Create a reactive form in the `ngOnInit()` method of the component.

   ```typescript
   export class ProductFormComponent {
     productForm: FormGroup;

     constructor(private fb: FormBuilder) {}

     ngOnInit() {
       this.productForm = this.fb.group({
         name: ['', [Validators.required, Validators.minLength(5)]],
         price: ['', [Validators.required, Validators.min(0)]],
         description: ['', [Validators.required, Validators.minLength(10)]]
       });
     }
   }
   ```

4. **Add Validation Messages:**

   Add validation messages to the component template.

   ```html
   <form [formGroup]="productForm" (ngSubmit)="onSubmit()">
     <div>
       <label for="name">Name</label>
       <input id="name" formControlName="name">
       <span *ngIf="productForm.get('name')?.invalid && productForm.get('name')?.touched">
         Name is required and must be at least 5 characters long.
       </span>
     </div>
     <div>
       <label for="price">Price</label>
       <input id="price" formControlName="price">
       <span *ngIf="productForm.get('price')?.invalid && productForm.get('price')?.touched">
         Price is required and must be greater than or equal to 0.
       </span>
     </div>
     <div>
       <label for="description">Description</label>
       <textarea id="description" formControlName="description"></textarea>
       <span *ngIf="productForm.get('description')?.invalid && productForm.get('description')?.touched">
         Description is required and must be at least 10 characters long.
       </span>
     </div>
     <button type="submit" [disabled]="productForm.invalid">Submit</button>
   </form>
   ```

5. **Handle Form Submission:**

   Handle the form submission in the `onSubmit()` method.

   ```typescript
   onSubmit() {
     if (this.productForm.valid) {
       console.log(this.productForm.value);
     }
   }
   ```

6. **Add the Component to the App Module:**

   Finally, add the new component to the `AppModule` so that it can be used in the application.

   ```typescript
   import { NgModule } from '@angular/core';
   import { BrowserModule } from '@angular/platform-browser';

   import { AppComponent } from './app.component';
   import { ProductFormComponent } from './product-form/product-form.component';

   @NgModule({
     declarations: [
       AppComponent,
       ProductFormComponent
     ],
     imports: [
       BrowserModule
     ],
     providers: [],
     bootstrap: [AppComponent]
   })
   export class AppModule { }
   ```

7. **Use the Component in the App Component:**

   Use the new component in the `AppComponent`.

   ```html
   <h1>Product Form</h1>
   <product-form></product-form>
   ```

8. **Run the Application:**

   Run the application to see the form in action.

   ```bash
   ng serve
   ```

   The following image shows the form in action.

   ![Product form](assets/images/product-form.png)

For more information on Angular's Reactive Forms, see the [Reactive Forms guide].

**Time taken:** 12.00257420539856 seconds

In [6]:
import os
import json

def merge_responses_to_markdown(response_dir, output_file):
    with open(output_file, "w") as outfile:
        for filename in sorted(os.listdir(response_dir)):
            if filename.endswith(".json"):
                with open(os.path.join(response_dir, filename), "r") as infile:
                    response_data = json.load(infile)
                    outfile.write(f"**User Prompt:** \n\n {response_data['prompt']}\n\n")
                    outfile.write(f"**Model Response:** \n\n {response_data['response']}\n\n")
                    outfile.write(f"**Time taken:** {response_data['time_taken']} seconds\n")
                    outfile.write("\n---\n\n")  # Separator between responses

# Example usage
merge_responses_to_markdown("prompt_responses", "output.md")